In [ ]:
# import the normal stuff
import tensorflow        as tf
import numpy             as np
import matplotlib.pyplot as plt

In [ ]:
# DO THIS ONLY IF YOU ARE IN COLAB
# Clone the repository with the TensorFlow2 update to StyleGAN2
# Thanks Alberto Rosas!!
%cd /content
!git clone https://github.com/ruberad/StyleGAN2-TensorFlow-2.x.git stylegan2
%cd /content/stylegan2

In [ ]:
# import StyleGAN2-specific stuff
from utils.utils_stylegan2 import convert_images_to_uint8
from stylegan2_generator   import StyleGan2Generator

In [ ]:
!nvidia-smi -L

In [ ]:
#impl = 'cuda' # 'ref' if cuda is not available in your machine
#gpu = True # False if tensorflow cpu is used
impl = 'ref'
gpu = False

In [ ]:
import gdown
# This is the URL of the online file with weights that StyleGan2 needs for generating faces
url = 'https://drive.google.com/uc?id=1afMN3e_6UuTTPDL63WHaA0Fb9EQrZceE'
# This is where we want the file to go (path on the virtual machine)
out = 'weights/ffhq.npy'

#gdown.download(url, out, quiet=False)

In [ ]:
weights_name='ffhq' # face generation, high quality
sg2 = StyleGan2Generator(weights=weights_name, impl=impl, gpu=gpu)

In [ ]:
seed = 1234
rng = np.random.RandomState(seed)
z = rng.randn(1, 512).astype('float32')
dlatents = sg2.mapping_network(z)

trunc = 0.7
w_avg = sg2.dlatent_avg
dlatents = w_avg + (dlatents - w_avg) * trunc

out = sg2.synthesis_network(dlatents)

img = convert_images_to_uint8(out, nchw_to_nhwc=True, uint8_cast=True)


In [ ]:
#plotting images
fig = plt.figure()
ax = plt.gca()
ax.axis('off')
img_plot = ax.imshow(img.numpy()[0])